In [1]:
# Generate demand for simulation analysis

In [30]:
import itertools
import numpy as np
from scipy.stats import norm
from scipy.stats import poisson
from sklearn import datasets
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [72]:
# Consider two rate classes for simplicity
n_class = 2
# Consider Length-of-Stay (los) of upto three days
los = 3
capacity = 50
# Consider two demand scenario, by changing capacity
demand_intensity = np.array([0.9, 1.5])
# Slope for linear demand curve
slope = np.array([-0.1, -0.15])
# Nightly rates for Monday stay night through Sunday stay night
# Higher weekday rates imply business hotel settings and higher weekend rates imply resorts
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

In [73]:
# Take the slope of -0.1 for rate class 1, then the mean true demand point for any arrival date/lengthof-
# stay combination is passed through by the line defined by this slope; 30 points on the price 
# axis for each arrival date/length-of-stay combination over the first 21 simulation days 
# are randomly generated from the interval (init_rate-50, init_rate+50)
# Sunday-one night stay as an example
drawsize = 30
# Random draw for one night stay for each day of week
randomRates_one = [np.random.randint(rates_init[i, j]-50, rates_init[i, j]+50+1, drawsize) 
                     for i, j in itertools.product(range(n_class), range(7))]
randomRates_one = np.array(randomRates_one).reshape(n_class, 7, drawsize)
# Check the averages are close to rates_init
np.round(np.average(randomRates_one, axis = 2), decimals=1)

array([[131.4, 136. , 133.5, 141.3, 141. , 105.9, 115.1],
       [108.7,  99.5, 105.8, 110. , 119. ,  98.5,  90.8]])

In [74]:
randomRates_one.shape

(2, 7, 30)

In [75]:
random_rates1_one

array([[170, 105, 177, 184, 116, 138, 113, 177, 133, 109, 139, 166,  92,
        153,  98, 185, 149, 130, 151, 117, 133, 161, 150, 123, 121, 115,
        161, 121, 173, 185],
       [162, 107, 140, 121, 148, 175,  95, 142, 156, 182, 110, 148, 181,
        138, 132, 157, 128, 106, 158, 106, 124, 128, 158, 168, 180, 151,
         89, 116,  98, 177],
       [113, 159, 129, 123, 116, 156, 140, 169, 127, 147,  92, 146, 147,
         87, 154, 169, 102, 106, 144,  92, 161, 137, 113, 161, 159, 141,
        106, 147, 119,  99],
       [139, 176, 116, 150, 105, 123, 129, 116, 176, 182, 100, 138, 144,
        131, 146,  98, 144, 178, 139, 121, 106,  96, 127,  90,  86,  86,
        136, 132,  95, 182],
       [148, 113, 141, 183, 122, 109, 105, 178, 150, 173, 132, 161, 178,
        106,  99, 130, 136, 120, 138, 139,  86, 174, 161,  99,  96, 111,
        102,  87, 144, 145],
       [ 67,  80, 104,  71, 129,  72, 146, 126, 122, 133,  92,  88, 120,
        133, 144, 146,  87, 131, 123, 137, 123,  61,

In [80]:
# Construct rates for more than one night stay, up to 3-night stay
# Sunday night arrival, two night stay rates equal Sunday one night + Monday one night stay
randomRates_two = [randomRates_one[i, j%7] 
                     + randomRates_one[i, (j+1)%7] 
                     for i, j in itertools.product(range(n_class), range(7))]

randomRates_three = [randomRates_one[i, j%7] 
                       + randomRates_one[i, (j+1)%7] 
                       + randomRates_one[i, (j+2)%7] 
                     for i, j in itertools.product(range(n_class), range(7))]

In [87]:
# For each arrival date, los can be 1-night, 2-night, or 3-night.
# Accordingly, the rates will be corresponding to different los.
# e.g., for Mon arrival, 2-night stay, the rate equals Mon single night rate + Tu single night rate
randomRates_one = np.array(randomRates_one).reshape(n_class, 7, drawsize)
randomRates_two = np.array(randomRates_two).reshape(n_class, 7, drawsize)
randomRates_three = np.array(randomRates_three).reshape(n_class, 7, drawsize)
randomRates = [(randomRates_one[i, j], randomRates_two[i, j], randomRates_three[i, j]) 
               for i, j in itertools.product(range(n_class), range(7))]

In [92]:
# axis 0 = n_class, axis 1 = day of week, axis 2 = los, and axis 3 = random draws
randomRates = np.array(randomRates).reshape(n_class, 7, los, drawsize)
randomRates.shape

(2, 7, 3, 30)

In [93]:
# Randomly generated rates for class 1, Sunday arrivals
randomRates[0, 0]

array([[ 91, 158, 114,  90, 110,  99, 126,  89, 161, 117,  86, 164, 135,
        124, 162, 121, 156, 100, 161, 118, 161, 180, 165, 165,  91, 155,
         94, 130, 175, 144],
       [197, 315, 247, 267, 243, 215, 299, 201, 297, 280, 224, 332, 225,
        286, 249, 251, 249, 245, 289, 303, 343, 315, 259, 331, 205, 300,
        265, 233, 310, 248],
       [333, 443, 345, 373, 352, 400, 470, 323, 428, 381, 395, 418, 332,
        454, 382, 342, 373, 383, 454, 442, 500, 500, 361, 489, 359, 468,
        361, 391, 406, 371]])

In [96]:
# Check accuracy, result should be similar to init_rate
np.mean(randomRates[1], axis = 2)

array([[108.73333333, 208.2       , 313.96666667],
       [ 99.46666667, 205.23333333, 315.26666667],
       [105.76666667, 215.8       , 334.76666667],
       [110.03333333, 229.        , 327.5       ],
       [118.96666667, 217.46666667, 308.3       ],
       [ 98.5       , 189.33333333, 298.06666667],
       [ 90.83333333, 199.56666667, 299.03333333]])

In [97]:
# Construct rates for more than one night stay, up to 3-night stay
rates_init

array([[135, 135, 135, 135, 135, 108, 108],
       [115, 115, 115, 115, 115,  92,  92]])

In [98]:
# Calculate averate rates for each arrival day of week and los combination
rates_arrival_los = [[rates_init[i, j],
                      rates_init[i, j] + rates_init[i, (j+1)%7],
                      rates_init[i, j] + rates_init[i, (j+1)%7] + rates_init[i, (j+2)%7]] 
                      for i, j in itertools.product(range(n_class), range(7))]
# Store it as a numpy array
rates_arrival_los = np.array(rates_arrival_los).reshape(n_class, 7, los)

In [104]:
rates_arrival_los

array([[[135, 270, 405],
        [135, 270, 405],
        [135, 270, 405],
        [135, 270, 378],
        [135, 243, 351],
        [108, 216, 351],
        [108, 243, 378]],

       [[115, 230, 345],
        [115, 230, 345],
        [115, 230, 345],
        [115, 230, 322],
        [115, 207, 299],
        [ 92, 184, 299],
        [ 92, 207, 322]]])

In [101]:
half_demand, slope

(array([22.5, 37.5]), array([-0.1 , -0.15]))

In [119]:
# Calculate y-intercepts, assuming los distribution for a stay night is 1/3, 1/3, and 1/3
# There exist one 1-night stay, two 2-night stays and three 3-night stays that cover 
# stay night in question.
# Each rate class contribute half of the demand for each stay night
half_demand = 0.5 * capacity * demand_intensity
# Intercepts for 1-night stay
A1 = np.array([[1, 0, 0, 0, 0, 0, 0],
               [0, 1, 0, 0, 0, 0, 0],
               [0, 0, 1, 0, 0, 0, 0],
               [0, 0, 0, 1, 0, 0, 0],
               [0, 0, 0, 0, 1, 0, 0],
               [0, 0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 0, 1]])
b1 = [(half_demand[0] * 1/3 - slope[i] * rates_arrival_los[i, j, 0]) 
               for i, j in itertools.product(range(n_class), range(7))]
b1 = np.array(b1).reshape(n_class, 7)
x1 = [np.linalg.solve(A1, b1[i]) for i in range(n_class)]

# Intercepts for 2-night stay
A2 = np.array([[1, 0, 0, 0, 0, 0, 1],
               [1, 1, 0, 0, 0, 0, 0],
               [0, 1, 1, 0, 0, 0, 0],
               [0, 0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 1, 0, 0],
               [0, 0, 0, 0, 1, 1, 0],
               [0, 0, 0, 0, 0, 1, 1]])
b2 = [(half_demand[0] * 1/3 - slope[i] * (rates_arrival_los[i, j, 1]
                                               +rates_arrival_los[i, (j-1+7)%7, 1])) 
               for i, j in itertools.product(range(n_class), range(7))]
b2 = np.array(b2).reshape(n_class, 7)
x2 = [np.linalg.solve(A2, b2[i]) for i in range(n_class)]

# Intercepts for 3-night stay
A3 = np.array([[1, 0, 0, 0, 0, 1, 1],
               [1, 1, 0, 0, 0, 0, 1],
               [1, 1, 1, 0, 0, 0, 0],
               [0, 1, 1, 1, 0, 0, 0],
               [0, 0, 1, 1, 1, 0, 0],
               [0, 0, 0, 1, 1, 1, 0],
               [0, 0, 0, 0, 1, 1, 1]])
b3 = [(half_demand[0] * 1/3 - slope[i] * (rates_arrival_los[i, j, 2]
                                               +rates_arrival_los[i, (j-1+7)%7, 2]
                                               +rates_arrival_los[i, (j-2+7)%7, 2]))
             for i, j in itertools.product(range(n_class), range(7))]
b3 = np.array(b3).reshape(n_class, 7)
x3 = [np.linalg.solve(A3, b3[i]) for i in range(n_class)]

In [161]:
# Create intercepts matrix for all arrival day-los combinations
# Total 2 * 7 * 3 = 42 intercepts
intercepts = np.concatenate((np.array(x1), np.array(x2), np.array(x3)), axis=1)
intercepts = np.reshape(intercepts, (n_class, los, 7), order='C')
intercepts = np.array([np.transpose(intercepts[i]) for i in range(n_class)])

In [162]:
intercepts

array([[[21.  , 30.75, 43.  ],
        [21.  , 30.75, 43.  ],
        [21.  , 30.75, 43.  ],
        [21.  , 30.75, 40.3 ],
        [21.  , 28.05, 37.6 ],
        [18.3 , 25.35, 37.6 ],
        [18.3 , 28.05, 40.3 ]],

       [[24.75, 38.25, 54.25],
        [24.75, 38.25, 54.25],
        [24.75, 38.25, 54.25],
        [24.75, 38.25, 50.8 ],
        [24.75, 34.8 , 47.35],
        [21.3 , 31.35, 47.35],
        [21.3 , 34.8 , 50.8 ]]])

In [164]:
# Mean of Poisson demand that is generated by the slope and y-intercept
# One mean demand for each of the 30 randomly drawn price
meanDemand = [intercepts[i,j, k] + slope[i] * randomRates[i, j, k, l] 
                    for i, j, k, l in itertools.product(range(n_class), range(7), range(los), range(drawsize))]
meanDemand = np.array(meanDemand).reshape(n_class, 7, los, drawsize)
zeros = np.zeros(n_class * 7 * los * drawsize).reshape(n_class, 7, los, drawsize)
# Can't have megative demand, so truncate mean demand at zero
meanDemand = np.maximum(meanDemand, zeros)

In [198]:
# Mean demand rate for a Sunday arrival (including 1-, 2-, and 3-night stay) 
np.average(meanDemand[0, 0], axis=1)

array([7.86      , 4.34333333, 4.07      ])

In [202]:
# Mean demand rate for a Sunday arrival (including 1-, 2-, and 3-night stay) 
meanDemand[0, 0, :, 0]

array([11.9 , 11.05,  9.7 ])

In [172]:
# Check total mean demand for each day of week for each random drawn price
totalDemand = [np.sum(meanDemand[i, j,:,k]) 
                     + np.sum(meanDemand[i, (j-1+7)%7, 1:, k]) 
                     + np.sum(meanDemand[i, (j-2+7)%7, 2:, k]) 
                     for i, j, k in itertools.product(range(n_class), range(7), range(drawsize))]
# Check total mean demand for each day of week
# This mean demand should be around half of the expected demand
totalDemand = np.array(totalDemand).reshape(n_class, 7, drawsize)
np.mean(totalDemand, axis=2)

array([[26.50166667, 26.43333333, 26.10833333, 23.85666667, 23.88      ,
        25.29333333, 23.95333333],
       [39.96166667, 46.99833333, 44.13833333, 37.09833333, 29.01666667,
        27.34      , 32.21666667]])

In [17]:
# Check total mean demand for each day of week for each random drawn price
meanDemand_rate1
totalDemand = [[np.sum(meanDemand_rate1[i,:,j]) 
                     + np.sum(meanDemand_rate1[(i-1+7)%7, 1:, j]) 
                     + np.sum(meanDemand_rate1[(i-2+7)%7, 2:, j]) 
                     for i in range(7)] for j in range(drawsize)]

In [18]:
# Check total mean demand for each day of week
# This mean demand should be around half of the expected demand
totalDemand_rate1 = np.array(totalDemand_rate1).reshape(7, drawsize)
np.mean(totalDemand_rate1, axis=1)

array([23.55      , 23.20166667, 17.575     , 27.73666667, 31.18666667,
       31.01666667, 27.53333333])

In [19]:
# Generate random demand for each day of week and length of stay combination
randomDemand_rate1 = [poisson.rvs(mu, size=1) for mu in np.nditer(meanDemand_rate1)]
randomDemand_rate1 = np.array(randomDemand_rate1).reshape(7, los, drawsize)

In [20]:
# Sunday arrivals for 1-night, 2-night, and 3-night stays
randomDemand_rate1[0]

array([[ 7, 11,  1,  3,  7,  3, 14,  4, 16, 13,  5, 12, 16,  7,  8,  2,
         9,  5, 10,  8,  9,  7,  7,  3, 11, 10,  5, 11,  2,  1],
       [ 0, 11,  0,  0,  5,  0,  7,  0,  2,  2,  7,  0,  4,  0,  5,  0,
         2,  6,  0,  8,  5,  1,  0,  1,  0,  6,  2,  9,  4,  0],
       [ 0,  5,  0,  1,  8,  0,  3,  0,  1,  0, 13,  0,  0,  5,  5,  0,
         5, 11,  0, 13,  1,  0,  1,  0,  0,  2, 10,  4,  2,  0]])

In [24]:
# Check number of total arrivals for each day of week, considering up to 3-night stay
arrivals = np.array([[np.sum(randomDemand_rate1[i,:,j]) 
                     + np.sum(randomDemand_rate1[(i-1+7)%7, 1:, j]) 
                     + np.sum(randomDemand_rate1[(i-2+7)%7, 2:, j]) 
                     for i in range(7)] for j in range(drawsize)])
np.average(arrivals, axis = 0)

array([23.7       , 24.1       , 28.13333333, 29.56666667, 27.13333333,
       23.46666667, 24.6       ])

In [53]:
randomRates1[0, 0], randomDemand_rate1[0, 0]

(array([170, 105, 177, 184, 116, 138, 113, 177, 133, 109, 139, 166,  92,
        153,  98, 185, 149, 130, 151, 117, 133, 161, 150, 123, 121, 115,
        161, 121, 173, 185]),
 array([ 7, 11,  1,  3,  7,  3, 14,  4, 16, 13,  5, 12, 16,  7,  8,  2,  9,
         5, 10,  8,  9,  7,  7,  3, 11, 10,  5, 11,  2,  1]))

In [50]:
# Create linear regression object
slopes_rate1_update = []
intercepts_rate1_update = []
regr = LinearRegression()
for i,j in itertools.product(range(7), range(los)):
    regr.fit(randomRates1[i, j, :].reshape(drawsize, -1), randomDemand_rate1[i, j, :])
    intercepts_rate1_update.append(regr.intercept_)
    slopes_rate1_update.append(regr.coef_)

In [51]:
slopes_rate1_update = np.array(slopes_rate1_update).reshape(7, los)
intercepts_rate1_update = np.array(intercepts_rate1_update).reshape(7, los)

In [52]:
slopes_rate1_update, intercepts_rate1_update

(array([[-0.10059067, -0.07400144, -0.07324504],
        [-0.10040019, -0.07948722, -0.08769441],
        [-0.07392731, -0.08347017, -0.06736604],
        [-0.09295855, -0.09159165, -0.06706168],
        [-0.10934749, -0.08273451, -0.06921872],
        [-0.13191376, -0.0654306 , -0.05958809],
        [-0.09095736, -0.06224823, -0.07314297]]),
 array([[21.8002463 , 23.68453917, 33.24287657],
        [20.45910642, 25.33949756, 39.42920389],
        [17.82753575, 26.36913056, 31.02874689],
        [19.27766284, 29.07564114, 29.04346101],
        [21.38907096, 23.82559093, 27.36577547],
        [21.49561654, 17.82119494, 24.14376591],
        [17.34752621, 19.22109644, 31.97463246]]))